In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
df = pd.read_csv('/content/sample_data/creditcard_2023.csv')
print(df.head())

In [ ]:
# Split the dataset based on the 'Class' column
class_0 = df[df['Class'] == 0]
class_1 = df[df['Class'] == 1]

# Display the first few rows of each subset to confirm the split
print("Class 0:")
print(class_0.head())

print("\nClass 1:")
print(class_1.head())

In [ ]:
# Sample 25,000 instances from each class
sample_class_0 = class_0.sample(n=25000, replace=True, random_state=42)
sample_class_1 = class_1.sample(n=25000, replace=True, random_state=42)

# Combine the samples into a single dataframe
sampled_df = pd.concat([sample_class_0, sample_class_1])

# Shuffle the combined dataframe
sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the first few rows of the sampled dataframe
print(sampled_df.head())

In [ ]:
# View information about the DataFrame
print(sampled_df.info())

In [ ]:
print('Dataset: ', sampled_df.shape)
sampled_df.sample(3)

In [ ]:
sampled_df['Class'].value_counts()

Observations-

*   We have 50000 rows of observations having 31 columns.
*   Class feature is balanced.
*   Class is the feature that indicates whether the transaction is fraudulent (1) or not (0).

# Data Processing

In [ ]:
# Checking missing values
sns.heatmap(sampled_df.isnull(), cbar=False)
plt.show()

Observations:

*   No missing value.

In [ ]:
duplicates = sampled_df.duplicated(keep=False)
duplicate_rows = sampled_df[duplicates]
print(duplicate_rows)

In [ ]:
# Drop Duplicates
sampled_df.drop_duplicates(inplace=True)
sampled_df['Class'].value_counts()

## EDA

In [ ]:
# Display summary statistics
sampled_df.describe()

In [ ]:
def single_plot_distribution(Class, sampled_df):
    # Get the value counts of the specified column
    value_counts = sampled_df[Class].value_counts()

    # Set up the figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5), gridspec_kw={'width_ratios': [1, 1]})

    # Donut pie chart
    pie_colors = sns.color_palette("Set2", n_colors=3)  # Set2 palette with 3 colors
    ax1.pie(value_counts, autopct='%0.001f%%', startangle=90, pctdistance=0.85, colors=pie_colors, labels=None)
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    ax1.add_artist(centre_circle)
    ax1.set_title(f'Distribution of {Class}', fontsize=16)

    # Bar chart
    bar_colors = sns.color_palette("Set2", n_colors=len(value_counts))  # Adjusted to the number of unique values
    sns.barplot(x=value_counts.index, y=value_counts.values, ax=ax2, palette=bar_colors)
    ax2.set_title(f'Count of {Class}', fontsize=16)
    ax2.set_xlabel(Class, fontsize=14)
    ax2.set_ylabel('Count', fontsize=14)

    # Rotate x-axis labels for better readability
    ax2.tick_params(axis='x', rotation=45)

    # Show the plots
    plt.tight_layout()
    plt.show()

# Class Distribution
single_plot_distribution('Class', sampled_df)

In [ ]:
# Cols to Plot
columns_to_plot =['V1', 'V2', 'V3', 'V4','Class']

# Data Columns
data_to_plot = sampled_df[columns_to_plot]

# Create a dictionary to map colors to unique values of the 'Class' column
Q_colors = {0: sns.color_palette("Set2")[0], 1: sns.color_palette("Set2")[1]}

# Creating the pairplot with the specified palette fo r categorical variables
sns.pairplot(data_to_plot, hue='Class', palette=Q_colors)
plt.show()

In [ ]:
# Num_COLS
NUM_COLS = sampled_df.select_dtypes(include=[np.number]).columns

# Define the number of rows and columns for subplots
num_rows = 5  # 4 rows
num_cols = 5  # 4 columns

# Create subplots with appropriate titles
fig, axes = plt.subplots(num_rows, num_cols, figsize=(25, 17))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Loop through each numerical column and create a box plot
for i, col in enumerate(NUM_COLS[:num_rows * num_cols]):
    sns.boxplot(x=sampled_df[col], ax=axes[i], color='skyblue')
    axes[i].set_title(col)

# Hide empty subplots
for i in range(len(NUM_COLS), num_rows * num_cols):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

In [ ]:
# Sample skewness calculation and sorting
skewness = sampled_df.skew().sort_values()

# Plotting the skewness
plt.figure(figsize=(14, 8))  # Adjusted figure size for better visibility
barplot = sns.barplot(x=skewness.index, y=skewness.values, palette='viridis')
plt.title('Skewness of Features in Sampled DataFrame')
plt.xlabel('Features')
plt.ylabel('Skewness')
plt.xticks(rotation=90)

# Annotate each bar with its skewness value
for i, v in enumerate(skewness.values):
    barplot.text(i, v, f'{v:.2f}', ha='center', va='bottom', fontsize=9)  # Adjusted font size

plt.show()

**Observations:**
* Features like V7, V20 are highly negative skewed.
* Features like V5, V28 are highly positive skewed.

In [ ]:
# Skewed features
skewed_features = ['V5', 'V7', 'V20', 'V28']

# Create subplots for boxplots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Customize colors
colors = sns.color_palette("Set2")

# Plot boxplots for each skewed feature
for i, feature in enumerate(skewed_features):
    row = i // 2
    col = i % 2
    sns.boxplot(x=feature, data=df, ax=axes[row, col], color=colors[i])
    axes[row, col].set_title(f'Boxplot of {feature}', fontsize=12)
    axes[row, col].set_xlabel('')
    axes[row, col].set_ylabel('')

# Set overall title and adjust layout
plt.suptitle('Boxplots of Skewed Features', size=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# Plot correlation matrix
corr_df = sampled_df.corr(method = 'spearman').round(2)
mask = np.triu(np.ones_like(corr_df, dtype=bool))
plt.figure(figsize=[25,15])
sns.heatmap(corr_df, cmap=sns.color_palette("magma", as_cmap = True), annot=True, mask = mask)
plt.show()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Drop the target variable ('Class') before calculating VIF
independent_variables = sampled_df.drop(columns=['Class'])

# Calculate VIF for each independent variable
vif_data = pd.DataFrame()
vif_data["Variable"] = independent_variables.columns
vif_data["VIF"] = [variance_inflation_factor(independent_variables.values, i) for i in range(independent_variables.shape[1])]

print(vif_data)

In [ ]:
# Sort the VIF data for better visualization
sorted_vif_data = vif_data.sort_values(by='VIF', ascending=False)

# Plotting
plt.figure(figsize=(10, 8))
plt.barh(sorted_vif_data['Variable'], sorted_vif_data['VIF'], color='skyblue')
plt.xlabel('VIF')
plt.ylabel('Variable')
plt.title('VIF Values')
plt.grid(axis='x')
plt.show()

### Observations:

* Variables like 'V17' and 'V16' have particularly high VIF values.
* Multicollinearity exists.

### Data Transformation

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Drop the 'id' column from the DataFrame
sampled_df_without_id = sampled_df.drop(columns=['id'])

# Extract features from DataFrame without 'id'
X = sampled_df_without_id.values

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize PCA
pca = PCA()

# Fit PCA to the standardized features
pca.fit(X_scaled)

# Determine the number of principal components to retain
explained_variance_ratio_cumulative = np.cumsum(pca.explained_variance_ratio_)
num_components_to_retain = np.argmax(explained_variance_ratio_cumulative >= 0.95) + 1

# Apply PCA transformation to the standardized features
pca = PCA(n_components=num_components_to_retain)
X_pca = pca.fit_transform(X_scaled)

# Convert X_pca to a DataFrame
X_pca_df = pd.DataFrame(X_pca, columns=[f'PC{i}' for i in range(1, num_components_to_retain+1)])
X_pca_df.head()

# Concatenate X_pca_df with the original DataFrame sampled_df_without_id
standardized_data_df = pd.concat([X_pca_df, sampled_df_without_id], axis=1)
standardized_data_df.head()


#Modeling

Features Selection

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [ ]:
x_data = df.iloc[:, 1:-1] # characteristic value
y_data = df.iloc[:, -1] # labels
# Partition dataset
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=42)
# Using ANOVA F-value as the scoring function to select the best 10 features
selector = SelectKBest(f_classif, k=10)
selector.fit(x_train, y_train)
# Obtain the index of the selected features
selected_features_indices = selector.get_support(indices=True)
# Obtain the name of the selected feature
selected_features_names = x_data.columns[selected_features_indices]
# Print selected feature names
print("Selected features:", selected_features_names)
# Draw rating charts for features
plt.bar(range(len(selector.scores_)), selector.scores_)
plt.xticks(range(len(selector.scores_)), x_data.columns, rotation=90)
plt.show()

In [ ]:
# Import necessary libraries
from sklearn.model_selection import train_test_split

# Define features and target variable

X = df[['V1', 'V2', 'V3', 'V4', 'V9', 'V10', 'V11', 'V12', 'V14', 'V16']]
y = df['Class']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.info()

In [ ]:
X_train.info()

Random Forest Classifier

In [ ]:
# Import necessary libraries
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Train a Random Forest classifier
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)

# Make predictions
y_pred = classifier.predict(X_test)

# Evaluate the model
RandomForest_accuracy = accuracy_score(y_test, y_pred)
RandomForest_conf_matrix = confusion_matrix(y_test, y_pred)
RandomForest_report = classification_report(y_test, y_pred)

print(f'Accuracy: {RandomForest_accuracy:.2f}')
print(f'Confusion Matrix:\n{RandomForest_conf_matrix}')
print(f'Classification Report:\n{RandomForest_report}')

Logistic Regression Classification

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Logistic Regression Analysis
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)

# Make predictions
y_pred = log_model.predict(X_test)

# Evaluate the model
LogisticRegression_accuracy = accuracy_score(y_test, y_pred)
LogisticRegression_conf_matrix = confusion_matrix(y_test, y_pred)
LogisticRegression_report = classification_report(y_test, y_pred)

print(f'Accuracy: {LogisticRegression_accuracy:.2f}')
print(f'Confusion Matrix:\n{LogisticRegression_conf_matrix}')
print(f'Classification Report:\n{LogisticRegression_report}')

Support Vector Classifier (SVM)

In [ ]:
from sklearn.svm import SVC

In [ ]:
# Support Vector Classifier (SVC) model
svc_model = SVC(kernel='linear')

# Train the SVC model
svc_model.fit(X_train, y_train)

# Make predictions
y_pred = svc_model.predict(X_test)

# Evaluate the model
svc_accuracy = accuracy_score(y_test, y_pred)
svc_conf_matrix = confusion_matrix(y_test, y_pred)
svc_report = classification_report(y_test, y_pred)

print(f'Accuracy: {svc_accuracy:.2f}')
print(f'Confusion Matrix:\n{svc_conf_matrix}')
print(f'Classification Report:\n{svc_report}')



```
# 此内容为代码格式
```

XGBClassifier

In [ ]:
from xgboost import XGBClassifier

# XGBoost Classifier model
xgb_model = XGBClassifier()

# Train the XGBoost Classifier model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
xgb_accuracy = accuracy_score(y_test, y_pred)
xgb_conf_matrix = confusion_matrix(y_test, y_pred)
xgb_report = classification_report(y_test, y_pred)

print(f'Accuracy: {xgb_accuracy:.2f}')
print(f'Confusion Matrix:\n{xgb_conf_matrix}')
print(f'Classification Report:\n{xgb_report}')

In [ ]:
# Create a dictionary with evaluation results
evaluation_data = {
    'Model': ['RandomForest', 'LogisticRegression', 'svc', 'xgb'],
    'Accuracy': [RandomForest_accuracy, LogisticRegression_accuracy, svc_accuracy,xgb_accuracy],
}

# Create a dataframe
evaluation_df = pd.DataFrame(evaluation_data)

# Sort the dataframe based on Accuracy and Precision columns in descending order
evaluation_df = evaluation_df.sort_values(by=['Accuracy'], ascending=False)

# Display the sorted dataframe
evaluation_df

#Deployment

In [ ]:
import pickle

# Save model artifact to local filesystem
# Save XGBoost model as xgboost.pkl
with open('xgboost.pkl', 'wb') as model_file:
  pickle.dump(xgb_model, model_file)

# Save Random Forest model as randomforest.pkl
with open('randomforest.pkl', 'wb') as model_file:
  pickle.dump(classifier, model_file)

# Save Support Vector Classifier model as supportvector.pkl
with open('supportvector.pkl', 'wb') as model_file:
  pickle.dump(svc_model, model_file)

# Save Logistic Regression model as logisticregression.pkl
with open('logisticregression.pkl', 'wb') as model_file:
  pickle.dump(log_model, model_file)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')